In [1]:
import os
from os.path import join
import sys 
import glob
import numpy as np
import nibabel as nib
from IPython.display import Image
from nipype.interfaces import fsl
from nipype.interfaces.fsl import utils
from nipype.testing import example_data
from subprocess import call

In [ ]:
call("hcpdownload.sh")

In [2]:
'''
Read the names and the data shapes
of all files with .nii format in 
the directory specified.
'''

def read_dir(diff): 
    for file in os.listdir(diff): 
        if '.nii' in file:
            data = nib.load(join(diff, file))
            data.uncache() #remove from the current memory so that the kernel doesn't die!
            print ('{:30}'.format(file), ' :' ,data.shape)

In [3]:
main_path = '/home/ubuntu/data/hcp/100307/'

In [4]:
fmri_path = join(main_path, 'rfMRI_REST1_LR/')
slice_path = join(fmri_path, 'split/vol0000.nii.gz')

In [5]:
t1_path =  join(join(main_path, 'T1w'), 'T1w_acpc_dc_restore_brain.nii.gz')

In [13]:
def fmri_slice(fmri, output_path):
    pass

In [6]:
def run_flirt(reference_path, template_path):
    flt = fsl.FLIRT(bins= 256, cost_func='corratio')
    flt.inputs.in_file = template_path
    flt.inputs.reference = reference_path
    flt.inputs.output_type = "NIFTI_GZ"
    flt.inputs.out_file = join(main_path, 'frmi_slice_to_T1w.nii.gz')
    flt.inputs.out_matrix_file = join(main_path, 'fmri_slice_to_T1w.mat')
    print('The following command is being run:')
    print(flt.cmdline) # doctest: +ELLIPSIS
    #'flirt -in structural.nii -ref mni.nii -out structural_flirt.nii.gz -omat structural_flirt.mat -bins 640 -searchcost mutualinfo'
    res = flt.run() #doctest: +SKIP

In [7]:
run_flirt(t1_path, slice_path)

The following command is being run:
flirt -in /home/ubuntu/data/hcp/100307/rfMRI_REST1_LR/split/vol0000.nii.gz -ref /home/ubuntu/data/hcp/100307/T1w/T1w_acpc_dc_restore_brain.nii.gz -out /home/ubuntu/data/hcp/100307/frmi_slice_to_T1w.nii.gz -omat /home/ubuntu/data/hcp/100307/fmri_slice_to_T1w.mat -bins 256 -searchcost corratio


In [12]:
def make_mask_from_label(labels, num, parc_affine): 
    masks_path = join(main_path, "masks")
    mask_id_num = 1*(labels == num)
    img = nib.Nifti1Image(mask_id_num, parc_affine)
    print('Storing file name: mask_id_{0}_T1w '.format(str(num)) + 'in path' + main_path)
    img.to_filename(os.path.join(main_path, 'mask_id_{0}_T1w'.format(str(num))))
    

In [13]:
def make_masks(labels, parc_affine): 
    
    for l in np.unique(labels): 
        make_mask_from_label(labels, int(l), parc_affine)

In [14]:
def load_affine(path): 
    
    aff = open(join(path, 'fmri_slice_to_T1w.mat'))
    aff = aff.read()
    aff = [x.split(' ')[:-1] for x in aff.split('\n')[:-1]]
    aff = np.array([[float(y) for y in x[::2]] for x in aff])
    print('The array for the transformation from fmri space to the T1 space')
    print(aff)
    return aff

In [15]:
def load_parcellation(path): 
    
    parc = nib.load(join(path, 'aparc+aseg.nii.gz'))
    print('Shape of the parcellaition file:', parc.shape)
    print('Affine matrix for parcellation file', parc.affine)
    labels = parc.get_fdata()
    print('Total number of different labels in parcellation file:', len(np.unique(labels)))
    return labels, parc.affine

In [16]:
affine = load_affine(main_path)
labels, parc_affine = load_parcellation(main_path)
make_masks(labels, parc_affine) 

The array for the transformation from fmri space to the T1 space
[[ 9.58394840e-01  1.04401833e-02  6.09542745e-03  2.24088513e+00]
 [-1.21214192e-02  9.08482330e-01  1.65524838e-03  1.21325548e+01]
 [ 8.24789862e-03 -9.00820793e-03  8.81513533e-01  6.91502496e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Shape of the parcellaition file: (260, 311, 260)
Affine matrix for parcellation file [[  -0.69999999    0.            0.           90.        ]
 [   0.            0.69999999    0.         -126.        ]
 [   0.            0.            0.69999999  -72.        ]
 [   0.            0.            0.            1.        ]]
Total number of different labels in parcellation file: 113
Storing file name: mask_id_0_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_2_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_4_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_5_T1w in path/home/ubuntu/data/hcp/100307/
Stori

Storing file name: mask_id_2027_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_2028_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_2029_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_2030_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_2031_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_2032_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_2033_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_2034_T1w in path/home/ubuntu/data/hcp/100307/
Storing file name: mask_id_2035_T1w in path/home/ubuntu/data/hcp/100307/


In [2]:
! cd masks
! ls -d "$PWD"/* > listofmasks.list
! cd ..